In [ ]:
!git clone "https://github.com/kakaobrain/jejueo.git"

In [ ]:
!git clone "https://github.com/pytorch/fairseq"

In [ ]:
!pip install sentencepiece

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

%cd '/content/gdrive/MyDrive/Tobigs/NLP_conference/'
%cd fairseq

In [ ]:
!pip install --editable ./

In [ ]:
cd /content/gdrive/MyDrive/Tobigs/NLP_conference/jejueo/translation

/content/gdrive/MyDrive/Tobigs/NLP_conference/jejueo/translation


# STEP 1. bpe segment for training

In [ ]:
!python bpe_segment.py --jit jit --vocab_size 4000

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=data/4k/bpe/bpe.train               --normalization_rule_name=identity               --model_prefix=data/4k/bpe/bpe               --character_coverage=0.995               --vocab_size=4000               --model_type=bpe
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/4k/bpe/bpe.train
  input_format: 
  model_prefix: data/4k/bpe/bpe
  model_type: BPE
  vocab_size: 4000
  self_test_sample_size: 0
  character_coverage: 0.995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_l

# STEP 2. fairseq-prepro

In [ ]:
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/gdrive/MyDrive/Tobigs/NLP_conference/fairseq/"

! echo $PYTHONPATH

/content
/env/python
/env/python:/content/gdrive/MyDrive/Tobigs/NLP_conference/fairseq/


In [ ]:
cd /content/gdrive/MyDrive/Tobigs/NLP_conference/jejueo/translation

/content/gdrive/MyDrive/Tobigs/NLP_conference/jejueo/translation


In [ ]:
!python prepro.py --src ko --tgt je --vocab_size 4000    # 표준어 -> 제주어

2022-06-07 12:15:34 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data/4k/ko-je-bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, simul_type=None, source_lan

# STEP 3. fairseq-train

In [ ]:
!fairseq-train data/4k/ko-je-bin \
    --arch transformer       \
    --optimizer adam \
    --lr 0.0005 \
    --label-smoothing 0.1 \
    --dropout 0.3       \
    --max-tokens 4000 \
    --stop-min-lr '1e-09' \
    --lr-scheduler inverse_sqrt       \
    --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy       \
    --max-epoch 1 \
    --warmup-updates 4000 \
    --warmup-init-lr '1e-07'    \
    --adam-betas '(0.9, 0.98)'       \
    --save-dir train/4k/ko-je-bin/ckpt  \
    --save-interval 1

2022-06-07 12:20:01 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': N

# STEP 4. fairseq-generate

In [ ]:
!fairseq-generate data/4k/ko-je-bin \
  --path train/4k/ko-je-bin/ckpt/checkpoint_best.pt \
  --source-lang ko --target-lang je \
  --valid-subset 'valid' \
  --gen-subset 'test' \
  --beam 5 \
  --remove-bpe 'sentencepiece'\
  --results-path prediction

2022-06-07 13:06:39 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name'

In [ ]:
!grep '^H' prediction/generate-test.txt | cut -f3- > prediction/gen.out.sys # 예측된 문장 (H)
!grep '^T' prediction/generate-test.txt | cut -f2- > prediction/gen.out.ref # 타겟(정답) 문장 (T)

# STEP6. Evaluation

In [ ]:
## 최종 BLEU score 확인
!fairseq-score \
--sys prediction/gen.out.sys \
--ref prediction/gen.out.ref

Namespace(ignore_case=False, order=4, ref='prediction/gen.out.ref', sacrebleu=False, sentence_bleu=False, sys='prediction/gen.out.sys')
BLEU4 = 73.24, 85.9/76.0/69.2/63.7 (BP=1.000, ratio=1.000, syslen=1834892, reflen=1835050)
